In [ ]:
import pandas as pd
from collections import Counter
import math
import re


# Load in the training set .csv
training_set = pd.read_csv("trg.csv")
training_set.head()


# Process the text, find a 'good model' with cross-validation
print("Text processing...")

#Extensions

for i in range(4000):
    for word in training_set['abstract'][i].split():
        if len(word) > 13 or len(word) <= 3:
            word.replace(word,'')
            
delete = ['1', '2', '3','4','5', '6', '7', '8', '9', '0', '-']
for i in range(4000):
    for word in training_set['abstract'][i].split():
        if word in delete:
            word.replace(word, '')
    
        



# Train the NBC with this data (your own NBC code)
def train(c, abstract):
    print("Training the NBC...")
    
    
    countA = 0
    countB = 0
    countE = 0
    countV = 0

    for i in range(4000):
        if c[i] == 'A':
            countA += 1
        elif c[i] == 'B':
            countB += 1
        elif c[i] == 'E':
            countE += 1
        elif c[i] == 'V':
            countV += 1



    wordsA = []
    wordsB = []
    wordsE = []
    wordsV = []

    for i in range(4000):
        for word in abstract[i].split():
            if c[i] == "A" and len(word) < 13:
                wordsA.append(word)
            elif c[i] == "B" and len(word) < 13:
                wordsB.append(word)
            elif c[i] == "E" and len(word) < 13:
                wordsE.append(word)
            elif c[i] == "V" and len(word) < 13:
                wordsV.append(word)

    all_words = []
    all_words = wordsA + wordsB + wordsE + wordsV
    unique = list(dict.fromkeys(all_words))
    #v = 32241
    v = len(unique)

    all_words.sort(key=Counter(all_words).get, reverse=True)
    unique = list(dict.fromkeys(all_words))

    delete_words = ['the', 'and', 'that', 'for','from', 'are', 'was','were', 'this', 'which', 'have', 'these', 'protein', 'with']

    #unique.sort(key = lambda s: len(s))
    #unique.reverse()
    unique = [x for x in unique if not any(c.isdigit() for c in x)]
    unique = [x for x in unique if len(x)>2]
    unique = [x for x in unique if len(x)<26]
    unique = [x for x in unique if x not in delete_words]


    unique = unique[:1000]


    pA = [1] * 1000
    pB = [1] * 1000
    pE = [1] * 1000
    pV = [1] * 1000

    for i in range(1000):
        pA[i] = (wordsA.count(unique[i]) + 1) / (countA + v)
        pB[i] = (wordsB.count(unique[i]) + 1) / (countB + v)
        pE[i] = (wordsE.count(unique[i]) + 1) / (countE + v)
        pV[i] = (wordsV.count(unique[i]) + 1) / (countV + v)

    priorA = countA / 4000
    priorB = countB / 4000
    priorE = countE / 4000
    priorV = countV / 4000


# Use this 'good model' to generate classifications. 
def classify(abstracts):
    
    # Text processing, cleaning, outlier removal, attribute selection etc. 
    # This function must be deterministic 
    # eg. if you select the 100 most frequent words, it must be the 100 most frequent words in the TRAINING set not
    # in the 'abstracts' parsed
    print("Processing the test abstracts...")
    
    
    # Run processed abstracts through the pre-trained naive bayes classifier
    print("Classifying the test abstracts...")
    
    results = ['E'] * 1000
    
    for i in range(1000):
        resA = countA / 4000
        resB = countB / 4000
        resE = countE / 4000
        resV = countV / 4000
        for word in abstracts[i].split():          
            if word in unique and len(word) > 2 and len(word) < 25:            
                resA *= pA[unique.index(word)]
                resB *= pB[unique.index(word)]
                resE *= pE[unique.index(word)]
                resV *= pV[unique.index(word)]
                                
                if resA >= resB and resA >= resE and resA >= resV:
                    results[i] = 'A'
                elif resB >= resA and resB >= resE and resB >= resV:
                    results[i] = 'B'
                elif resE >= resA and resE >= resB and resE >= resV:
                    results[i] = 'E'
                elif resV >= resA and resV >= resE and resV >= resB:
                    results [i] = 'V'
    
    
    return results
    
    
# Load in the test set .csv
test_set = pd.read_csv("tst.csv")

#train model
train(training_set['class'], training_set['abstract'])

# Apply the model to the test set
test_set_class_predictions = classify(test_set["abstract"])
test_set["class"] = test_set_class_predictions


# Write the test set classifications to a .csv so it can be submitted to Kaggle
test_set.drop(["abstract"], axis = 1).to_csv("tst_kaggle.csv", index=False)
test_set.head()



Text processing...
Training the NBC...
